<i>Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.</i>
<br>
# Hyperparameter Tuning for Matrix Factorization Using the Neural Network Intelligence Toolkit

This notebook shows how to use the **[Neural Network Intelligence](https://nni.readthedocs.io/en/latest/) toolkit (NNI)** for tuning hyperparameters of a matrix factorization model. In particular, we optimize the hyperparameters of [Surprise SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html).

NNI is a toolkit to help users design and tune machine learning models (e.g., hyperparameters), neural network architectures, or complex system’s parameters, in an efficient and automatic way. NNI has several appealing properties: ease of use, scalability, flexibility and efficiency. NNI comes with [several tuning algorithms](https://nni.readthedocs.io/en/latest/Builtin_Tuner.html) built in. It also allows users to [define their own general purpose tuners](https://nni.readthedocs.io/en/latest/Customize_Tuner.html). NNI can be executed in a distributed way on a local machine, a remote server, or a large scale training platform such as OpenPAI or Kubernetes. 

In this notebook we execute several NNI _experiments_ on the same data sets obtained from Movielens with a training-validation-test split. Each experiment corresponds to one of the built-in tuning algorithms. It consists of many parallel _trials_, each of which corresponds to a choice of hyperparameters sampled by the tuning algorithm. All the experiments require a call to the same [python script](../../reco_utils/nni/svd_training.py) for training the SVD model and evaluating rating and ranking metrics on the test data. This script has been adapted from the [Surprise SVD notebook](../02_model/surprise_svd_deep_dive.ipynb) with only a few changes. In all experiments, we maximize precision@10. 

For this notebook we use a _local machine_ as the training platform (this can be any machine running the `reco_base` conda environment). In this case, NNI uses the available processors of the machine to parallelize the trials, subject to the value of `trialConcurrency` we specify in the configuration. Our runs and the results we report were obtained on a [Standard_D16_v3 virtual machine](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/sizes-general#dv3-series-1) with 16 vcpus and 64 GB memory.

### 1. Global Settings

In [1]:
import sys
sys.path.append("../../")
import json
import os
import surprise
import papermill as pm
import pandas as pd
import shutil
import subprocess
import yaml
import pkg_resources
from tempfile import TemporaryDirectory

import reco_utils
from reco_utils.common.timer import Timer
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split
from reco_utils.evaluation.python_evaluation import rmse, precision_at_k, ndcg_at_k
from reco_utils.nni.nni_utils import (check_experiment_status, check_stopped, check_metrics_written, get_trials,
                                      stop_nni, start_nni)
from reco_utils.recommender.surprise.surprise_utils import compute_rating_predictions, compute_ranking_predictions

print("System version: {}".format(sys.version))
print("Surprise version: {}".format(surprise.__version__))
print("NNI version: {}".format(pkg_resources.get_distribution("nni").version))

tmp_dir = TemporaryDirectory()

%load_ext autoreload
%autoreload 2

System version: 3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]
Surprise version: 1.0.6
NNI version: 0.5.2.1


### 2. Prepare Dataset
1. Download data and split into training, validation and test sets
2. Store the data sets to a local directory.

In [2]:
# Parameters used by papermill
# Select Movielens data size: 100k, 1m
MOVIELENS_DATA_SIZE = '100k'
SURPRISE_READER = 'ml-100k'
TMP_DIR = tmp_dir.name
NUM_EPOCHS = 30
MAX_TRIAL_NUM = 100
# time (in seconds) to wait for each tuning experiment to complete
WAITING_TIME = 20
MAX_RETRIES = 200

In [3]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

4.93MB [00:01, 4.07MB/s]                            


,userID,itemID,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


In [4]:
train, validation, test = python_random_split(data, [0.7, 0.15, 0.15])

In [5]:
LOG_DIR = os.path.join(TMP_DIR, "experiments")
os.makedirs(LOG_DIR, exist_ok=True)

DATA_DIR = os.path.join(TMP_DIR, "data") 
os.makedirs(DATA_DIR, exist_ok=True)

TRAIN_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_train.pkl"
train.to_pickle(os.path.join(DATA_DIR, TRAIN_FILE_NAME))

VAL_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_val.pkl"
validation.to_pickle(os.path.join(DATA_DIR, VAL_FILE_NAME))

TEST_FILE_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_test.pkl"
test.to_pickle(os.path.join(DATA_DIR, TEST_FILE_NAME))

### 3. Prepare Hyperparameter Tuning 

We now prepare a training script [svd_training_nni.py](../../reco_utils/nni/svd_training.py) for the hyperparameter tuning, which will log our target metrics such as precision, NDCG, RMSE.
We define the arguments of the script and the search space for the hyperparameters. All the parameter values will be passed to our training script.<br>
Note that we specify _precision@10_ as the primary metric. We will also instruct NNI (in the configuration file) to _maximize_ the primary metric. This is passed as an argument in the training script and the evaluated metric is returned through the NNI python library. In addition, we also evaluate RMSE and NDCG@10.   

The `script_params` below are the parameters of the training script that are fixed (unlike `hyper_params` which are tuned). In particular, `VERBOSE, BIASED, RANDOM_STATE, NUM_EPOCHS` are parameters used in the [SVD method](../02_model/surprise_svd_deep_dive.ipynb) and `REMOVE_SEEN` removes the training data from the recommended items. 

In [6]:
EXP_NAME = "movielens_" + MOVIELENS_DATA_SIZE + "_svd_model"
PRIMARY_METRIC = "precision_at_k"
RATING_METRICS = ["rmse"]
RANKING_METRICS = ["precision_at_k", "ndcg_at_k"]  
USERCOL = "userID"
ITEMCOL = "itemID"
REMOVE_SEEN = True
K = 10
RANDOM_STATE = 42
VERBOSE = True
BIASED = True

script_params = " ".join([
    "--datastore", DATA_DIR,
    "--train-datapath", TRAIN_FILE_NAME,
    "--validation-datapath", VAL_FILE_NAME,
    "--surprise-reader", SURPRISE_READER,
    "--rating-metrics", " ".join(RATING_METRICS),
    "--ranking-metrics", " ".join(RANKING_METRICS),
    "--usercol", USERCOL,
    "--itemcol", ITEMCOL,
    "--k", str(K),
    "--random-state", str(RANDOM_STATE),
    "--epochs", str(NUM_EPOCHS),
    "--primary-metric", PRIMARY_METRIC
])

if BIASED:
    script_params += " --biased"
if VERBOSE:
    script_params += " --verbose"
if REMOVE_SEEN:
    script_params += " --remove-seen"

In [7]:
# hyperparameters search space
# We do not set 'lr_all' and 'reg_all' because they will be overriden by the other lr_ and reg_ parameters

hyper_params = {
    'n_factors': {"_type": "choice", "_value": [10, 50, 100, 150, 200]},
    'init_mean': {"_type": "uniform", "_value": [-0.5, 0.5]},
    'init_std_dev': {"_type": "uniform", "_value": [0.01, 0.2]},
    'lr_bu': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_bi': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_pu': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'lr_qi': {"_type": "uniform", "_value": [1e-6, 0.1]}, 
    'reg_bu': {"_type": "uniform", "_value": [1e-6, 1]},
    'reg_bi': {"_type": "uniform", "_value": [1e-6, 1]}, 
    'reg_pu': {"_type": "uniform", "_value": [1e-6, 1]}, 
    'reg_qi': {"_type": "uniform", "_value": [1e-6, 1]}
}

In [8]:
with open(os.path.join(TMP_DIR, 'search_space_svd.json'), 'w') as fp:
    json.dump(hyper_params, fp)

We also create a yaml file for the configuration of the trials and the tuning algorithm to be used (in this experiment we use the [TPE tuner](https://nni.readthedocs.io/en/latest/hyperoptTuner.html)). 

In [9]:
config = {
    "authorName": "default",
    "experimentName": "surprise_svd",
    "trialConcurrency": 8,
    "maxExecDuration": "1h",
    "maxTrialNum": MAX_TRIAL_NUM,
    "trainingServicePlatform": "local",
    # The path to Search Space
    "searchSpacePath": "search_space_svd.json",
    "useAnnotation": False,
    "logDir": LOG_DIR,
    "tuner": {
        "builtinTunerName": "TPE",
        "classArgs": {
            #choice: maximize, minimize
            "optimize_mode": "maximize"
        }
    },
    # The path and the running command of trial
    "trial":  {
      "command": sys.prefix + "/bin/python svd_training.py" + " " + script_params,
      "codeDir": os.path.join(os.path.split(os.path.abspath(reco_utils.__file__))[0], "nni"),
      "gpuNum": 0
    }
}
 
with open(os.path.join(TMP_DIR, "config_svd.yml"), "w") as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

### 4. Execute NNI Trials

The conda environment comes with NNI installed, which includes the command line tool `nnictl` for controlling and getting information about NNI experiments. <br>
To start the NNI tuning trials from the command line, execute the following command: <br>
`nnictl create --config <path of config_svd.yml>` <br>
In the cell below, we call this command programmatically. <br>
You can see the progress of the experiment by using the URL links output by the above command.

![](https://recodatasets.blob.core.windows.net/images/nn1.png)

![](https://recodatasets.blob.core.windows.net/images/nn2.png)

![](https://recodatasets.blob.core.windows.net/images/nn3.png)

In [10]:
# Make sure that there is no experiment running
stop_nni()

In [11]:
config_path = os.path.join(TMP_DIR, 'config_svd.yml')
nni_env = os.environ.copy()
nni_env['PATH'] = sys.prefix + '/bin:' + nni_env['PATH']
proc = subprocess.run([sys.prefix + '/bin/nnictl', 'create', '--config', config_path], env=nni_env)
if proc.returncode != 0:
    raise RuntimeError("'nnictl create' failed with code %d" % proc.returncode)

In [12]:
with Timer() as time_tpe:
    check_experiment_status(wait=WAITING_TIME, max_retries=MAX_RETRIES)

### 5. Show Results

The trial with the best metric and the corresponding metrics and hyperparameters can also be read from the Web UI

![](https://recodatasets.blob.core.windows.net/images/nni4.png)

or from the JSON file created by the training script. Below, we do this programmatically using [nni_utils.py](../../reco_utils/nni/nni_utils.py) 

In [13]:
check_metrics_written(wait=WAITING_TIME, max_retries=MAX_RETRIES)
trials, best_metrics, best_params, best_trial_path = get_trials('maximize')

In [14]:
best_metrics

{'rmse': 1.0271230641650002,
 'ndcg_at_k': 0.06958394898955478,
 'precision_at_k': 0.05867944621938233}

In [15]:
best_params

{'parameter_id': 34,
 'parameter_source': 'algorithm',
 'parameters': {'n_factors': 10,
  'init_mean': -0.49860893141386153,
  'init_std_dev': 0.19468245761524738,
  'lr_bu': 0.07308312633406484,
  'lr_bi': 9.16418742754133e-05,
  'lr_pu': 0.018101794279496805,
  'lr_qi': 0.018446906101458462,
  'reg_bu': 0.5851767617907916,
  'reg_bi': 0.2415361782986008,
  'reg_pu': 0.9794861282824148,
  'reg_qi': 0.8023432906929577}}

In [16]:
best_trial_path

'/tmp/tmpoenu23x6/experiments/H7Oq2UEL/trials/YProD'

This directory path is where info about the trial can be found, including logs, parameters and the model that was learned. To evaluate the metrics on the test data, we get the SVD model that was saved as `model.dump` in the training script.

In [17]:
svd = surprise.dump.load(os.path.join(best_trial_path, "model.dump"))[1]

The following function computes all the metrics given an SVD model.

In [18]:
def compute_test_results(svd):
    test_results = {}
    predictions = compute_rating_predictions(svd, test, usercol="userID", itemcol="itemID")
    for metric in RATING_METRICS:
        test_results[metric] = eval(metric)(test, predictions)

    all_predictions = compute_ranking_predictions(svd, train, usercol="userID", itemcol="itemID", remove_seen=REMOVE_SEEN)
    for metric in RANKING_METRICS:
        test_results[metric] = eval(metric)(test, all_predictions, col_prediction='prediction', k=K)
    return test_results

In [19]:
test_results_tpe = compute_test_results(svd)
print(test_results_tpe)

{'rmse': 1.0251101055610055, 'precision_at_k': 0.09722814498933904, 'ndcg_at_k': 0.11407488833199972}


### 6. More Tuning Algorithms
We now apply other tuning algorithms supported by NNI to the same problem. For details about these tuners, see the [NNI docs.](https://nni.readthedocs.io/en/latest/tuners.html#)
The only change needed is in the relevant entry in the configuration file.

In summary, the tuners used in this notebook are the following:
- Tree-structured Parzen Estimator (TPE), within the Sequential Model-Based Optimization (SMBO) framework,
- SMAC, also an instance of SMBO,
- Hyperband
- Metis, an implementation of Bayesian optimization with Gaussian Processes
- a Naive Evolutionary algorithm
- an Annealing method for sampling, and  
- plain Random Search as a baseline.  

For more details and references to the relevant literature, see the [NNI github](https://github.com/Microsoft/nni/blob/master/docs/en_US/Builtin_Tuner.md).

In [20]:
# Random search
config['tuner']['builtinTunerName'] = 'Random'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))
    print(config_path)

/tmp/tmpoenu23x6/config_svd.yml


In [21]:
stop_nni()
with Timer() as time_random:
    start_nni(config_path, wait=WAITING_TIME, max_retries=MAX_RETRIES)

In [22]:
check_metrics_written(wait=WAITING_TIME, max_retries=MAX_RETRIES)
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_random = compute_test_results(svd)

In [23]:
# Annealing
config['tuner']['builtinTunerName'] = 'Anneal'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [24]:
stop_nni()
with Timer() as time_anneal:
    start_nni(config_path, wait=WAITING_TIME, max_retries=MAX_RETRIES)

In [25]:
check_metrics_written(wait=WAITING_TIME, max_retries=MAX_RETRIES)
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_anneal = compute_test_results(svd)

In [26]:
# Naive evolutionary search
config['tuner']['builtinTunerName'] = 'Evolution'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [27]:
stop_nni()
with Timer() as time_evolution:
    start_nni(config_path, wait=WAITING_TIME, max_retries=MAX_RETRIES)

In [28]:
check_metrics_written(wait=WAITING_TIME, max_retries=MAX_RETRIES)
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_evolution = compute_test_results(svd)

The SMAC tuner requires to have been installed with the following command <br>
`nnictl package install --name=SMAC`

In [29]:
# SMAC
config['tuner']['builtinTunerName'] = 'SMAC'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [30]:
# Check if installed
proc = subprocess.run([sys.prefix + '/bin/nnictl', 'package', 'show'], stdout=subprocess.PIPE)
if proc.returncode != 0:
    raise RuntimeError("'nnictl package show' failed with code %d" % proc.returncode)
if 'SMAC' not in proc.stdout.decode().strip().split():
    proc = subprocess.run([sys.prefix + '/bin/nnictl', 'package', 'install', '--name=SMAC'])
    if proc.returncode != 0:
        raise RuntimeError("'nnictl package install' failed with code %d" % proc.returncode)

In [31]:
# Skipping SMAC optimization for now
# stop_nni()
with Timer() as time_smac:
#    start_nni(config_path, wait=WAITING_TIME, max_retries=MAX_RETRIES)
    pass


In [32]:
#check_metrics_written()
#svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
#test_results_smac = compute_test_results(svd)

In [33]:
# Metis
config['tuner']['builtinTunerName'] = 'MetisTuner'
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [34]:
stop_nni()
with Timer() as time_metis:
    start_nni(config_path, wait=WAITING_TIME, max_retries=MAX_RETRIES)

RuntimeError: NNI experiment failed to complete with status ERROR: ['Dispatcher error: Dispatcher Error: This socket has been ended by the other party']

In [ ]:
check_metrics_written()
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_metis = compute_test_results(svd)

Hyperband follows a different style of configuration from other tuners. See [the NNI documentation](https://nni.readthedocs.io/en/latest/hyperbandAdvisor.html). Note that the [training script](../../reco_utils/nni/svd_training.py) needs to be adjusted as well, since each Hyperband trial receives an additional parameter `STEPS`, which corresponds to the resource allocation _r<sub>i</sub>_ in the [Hyperband algorithm](https://arxiv.org/pdf/1603.06560.pdf). In this example, we used `STEPS` in combination with `R` to determine the number of epochs that SVD will run for in every trial.

In [ ]:
# Hyperband
config['advisor'] = {
  'builtinAdvisorName': 'Hyperband',
  'classArgs': {
    'R': NUM_EPOCHS,
    'eta': 3,
    'optimize_mode': 'maximize'
  }
}
config.pop('tuner')
with open(config_path, 'w') as fp:
    fp.write(yaml.dump(config, default_flow_style=False))

In [ ]:
stop_nni()
with Timer() as time_hyperband:
    start_nni(config_path, wait=WAITING_TIME, max_retries=MAX_RETRIES)

In [ ]:
check_metrics_written()
svd = surprise.dump.load(os.path.join(get_trials('maximize')[3], "model.dump"))[1]
test_results_hyperband = compute_test_results(svd)

In [ ]:
test_results_tpe.update({'time': time_tpe.interval})
test_results_random.update({'time': time_random.interval})
test_results_anneal.update({'time': time_anneal.interval})
test_results_evolution.update({'time': time_evolution.interval})
test_results_smac.update({'time': time_smac.interval})
test_results_metis.update({'time': time_metis.interval})
test_results_hyperband.update({'time': time_hyperband.interval})

In [ ]:
res_df = pd.DataFrame(index=['TPE', 'Random Search', 'Annealing', 'Evolution', 'SMAC', 'Metis', 'Hyperband'],
                      data=[res for res in [test_results_tpe, test_results_random, test_results_anneal, test_results_evolution, 
                                            test_results_smac, test_results_metis, test_results_hyperband]] 
                     )

In [ ]:
res_df.sort_values(by='precision_at_k', ascending=False).round(3)

As we see in the table above, _annealing_ performs best with respect to the primary metric (precision@10) that all the tuners optimized. Also the best NDCG@10 is obtained for annealing and correlates well with precision@10. RMSE on the other hand does not correlate well and is not optimized for annealing, since finding the top k recommendations in the right order is a different task from predicting ratings (high and low) accurately.     
We have also observed that the above ranking of the tuners is not consistent and may change when trying these experiments multiple times. Since some of these tuners rely heavily on randomized sampling, a larger number of trials is required to get more consistent metrics.
In addition, some of the tuning algorithms themselves come with parameters, which can affect their performance.

In [ ]:
# Stop the NNI experiment 
stop_nni()

In [ ]:
tmp_dir.cleanup()

### 7. Concluding Remarks

We showed how to tune **all** the hyperparameters accepted by Surprise SVD simultaneously, by utilizing the NNI toolkit. 
For example, training and evaluation of a single SVD model takes about 50 seconds on the 100k MovieLens data on a Standard D2_V2 VM. Searching through 100 different combinations of hyperparameters sequentially would take about 80 minutes whereas each of the above experiments took about 10 minutes by exploiting parallelization on a single D16_v3 VM. With NNI, one can take advantage of concurrency and multiple processors on a virtual machine and can use a variety of tuning methods to navigate efficiently through a large space of hyperparameters.<br>
For examples of scaling larger tuning workloads on clusters of machines, see [the notebooks](./README.md) that employ the [Azure Machine Learning service](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters).  

### References

* [Matrix factorization algorithms in Surprise](https://surprise.readthedocs.io/en/stable/matrix_factorization.html) 
* [Surprise SVD deep-dive notebook](../02_model/surprise_svd_deep_dive.ipynb)
* [Neural Network Intelligence toolkit](https://github.com/Microsoft/nni)